In [ ]:
from scipy.io.wavfile import read
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks
import pandas as pd
%matplotlib inline
mydir= '../freesound_lgbm/input/'
emo_length = 535

In [ ]:
def stft(sig, frameSize, overlapFac=0.384, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    
    # zeros at beginning (thus center of 1st window should be for sample nr. 0)
    samples = np.append(np.zeros(np.floor(frameSize/2.0).astype(int)), sig)  
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize).astype(int))
    
    
    frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win
    
    return np.fft.rfft(frames)    
    
""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))
    
    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    scale = scale.astype(int)
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,scale[i]:], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,scale[i]:scale[i+1]], axis=1)
    
    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[scale[i]:])]
        else:
            freqs += [np.mean(allfreqs[scale[i]:scale[i+1]])]
    
    return newspec, freqs

""" plot spectrogram"""
def plotstft(audiopath, binsize=2**9, plotpath=None, colormap="jet"):
    samplerate, samples = wav.read(audiopath)
    s = stft(samples, binsize)
    
    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel
    
    timebins, freqbins = np.shape(ims)
    
    #plt.figure(figsize=(15, 7.5))
    fig = plt.imshow(np.transpose(ims), origin="lower", aspect="auto", cmap=colormap, interpolation="none")
    #plt.colorbar()
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)

    plt.axis('off')

    if plotpath:
        plt.savefig(plotpath, pad_inches = -0.045, bbox_inches="tight")
        #plt.show()
    else:
        plt.show()
        
    plt.clf()

In [ ]:
import os
test_df = pd.read_csv(mydir+"sample_submission.csv")
train_df = pd.read_csv(mydir+ "train.csv")

s = set(train_df["label"])
for e in s:
    os.makedirs("train/"+e)


In [ ]:
import os
from scipy import signal
from random import shuffle
a = os.listdir(mydir+'audio_test')
saveSpectDirValidation= 'validation/'
saveSpectDirTrain= 'train/'
saveSpectDirTest= 'test/one/'
new_ind = 0
shuffle(a)
print len(a)
print train_df["label"][0]
print train_df["label"][np.argmax(train_df["fname"] == "00044347.wav")]

In [ ]:

def filltest():
    new_ind = 0
    for i in a:
        plotstft( mydir+'audio_test/'+i, plotpath =  saveSpectDirTest + i[:-4]+'.JPEG')
        new_ind+=1

def filltrain():
    for i in a:
        lab = train_df["label"][np.argmax(train_df["fname"] == i)]
        if(new_ind < len(a)*0.75):
            plotstft( mydir+'audio_train_trimmed/'+i, plotpath =  saveSpectDirTrain + lab + '/' + i[:-4]+'.JPEG')
        else:
            plotstft( mydir+'audio_train_trimmed/'+i, plotpath =  saveSpectDirValidation  + lab + '/' + i[:-4]+'.JPEG')

        new_ind+=1


In [ ]:
filltest()